In [ ]:
from brian2 import *
%matplotlib inline

## Now make the synapse filter a difference of exponentials defined implicitly by two coupled odes

## And slowly build the balanced network

In [ ]:
start_scope()

vt = 1
vr = 0

tau_e = 15*ms
tau_i = 10*ms
tau1 = 1 * ms
tau2_e = 3 * ms
tau2_i = 2 * ms
tau_scale = 1 * ms

eqs = '''
dv/dt = (mu-v)/tau + (I_e - I_i)/tau_scale : 1
dI_e/dt = -(I_e - x_e)/tau2_e : 1
dI_i/dt = -(I_i - x_i)/tau2_i : 1
dx_e/dt = -x_e / tau1 : 1
dx_i/dt = -x_i / tau1 : 1
mu : 1
tau : second
'''

NE = 4000
NI = 1000
N = NE + NI
G = NeuronGroup(N, eqs, threshold='v>vt', reset='v=vr', method='euler', refractory=5*ms)
G.v = np.random.rand(N) * (vt - vr) + vr
Pe = G[:NE]
Pi = G[NE:]

fac = 1.0
Pe.mu = fac * np.random.uniform(1.1, 1.2, NE) * (vt - vr) + vr
Pi.mu = fac * np.random.uniform(1.0, 1.05, NI) * (vt - vr) + vr
# for visualization
#G.mu[0] = 0
#G.mu[NE] = 0


Pe.tau = tau_e
Pi.tau = tau_i

See = Synapses(Pe, Pe, 'w : 1', on_pre='''x_e += w''')
See.connect(p=0.2)
See.w = 0.024 

Sii = Synapses(Pi, Pi, 'w : 1', on_pre='''x_i += w''')
Sii.connect(p=0.5)
Sii.w = 0.057

Sei = Synapses(Pi, Pe, 'w : 1', on_pre='''x_i += w''')
Sei.connect(p=0.5)
Sei.w = 0.045

Sie = Synapses(Pe, Pi, 'w : 1', on_pre='''x_e += w''')
Sie.connect(p=0.5)
Sie.w = 0.014

Mn = StateMonitor(G, ['v', 'I_e', 'I_i', 'x_e', 'x_i'], record=[0, NE])
sme = SpikeMonitor(Pe)
smi = SpikeMonitor(Pi)
rm = PopulationRateMonitor(Pi)

run(5000*ms)

In [ ]:
NE = 1
figure(figsize=(15, 8))
subplot(211)
plot(Mn.t/ms, Mn.v[0], label='vE', color='b')
plt.legend(loc=4)

plt.twinx()
plot(Mn.t/ms, Mn.I_e[0], label='NE, In_e', color='g')
plot(Mn.t/ms, Mn.I_i[0], label='NE, In_i', color='r')
legend()

subplot(212)
plot(Mn.t/ms, Mn.v[NE], label='vI', color='b')
plt.legend(loc=4)

plt.twinx()
plot(Mn.t/ms, Mn.I_e[NE], label='NI, In_e', color='g')
plot(Mn.t/ms, Mn.I_i[NE], label='NI, In_i', color='r')
legend(loc='best')
xlabel('Time (ms)');

In [ ]:
NE=1
fig = plt.figure(figsize=(15, 5))
ax = plt.subplot(111)
ax.plot(Mn.t/ms, Mn.v[NE], label='v', color='b')
ax.set_ylabel('voltage [a.u]', color='blue')
ax.tick_params(axis='y', colors='blue')
plt.legend(loc=4, prop={'size': 15})

plt.twinx()
plt.plot(Mn.t/ms, Mn.I_e[NE], label='$I_E$', color='g')
plt.plot(Mn.t/ms, Mn.I_i[NE], label='$I_I$', color='r')
plt.tick_params(axis='y', colors='red')
plt.ylabel('current [a.u.]', color='red')
plt.legend(prop={'size': 15});
plt.savefig('../../../talk/figures/balanced_currents.pdf')

In [ ]:
plt.figure(figsize=(15, 5))
mask = sme.i < 400
plot(sme.t[mask]/ms, sme.i[mask], '.b', markersize=2)
#plot(smi.t/ms, smi.i + NE, '.r', markersize=2)
xlabel('Time (ms)')
ylabel('Neuron index')
plt.tight_layout()
plt.savefig('../../../talk/figures/irregular_firing.pdf');

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(rm.t, rm.rate);